In [82]:
from pathlib import Path
from cloudpathlib import S3Path, S3Client
import nibabel as nib 
import neuropythy as ny
from nilearn import datasets, input_data, connectome
import numpy as np
from scipy.stats import pearsonr
import pandas as pd

# Make sure that we have a cache path:
cache_path = Path('/tmp/cache')
if not cache_path.exists():
    cache_path.mkdir()

hcp_base_path = S3Path(
    's3://hcp-openaccess/HCP_1200/',
    client=S3Client(
        local_cache_dir=cache_path,
        profile_name='hcp'))

In [83]:
hcp_subdirs = list(hcp_base_path.iterdir())
# There are about 1200 of these, so we won't show them all, just the first 10:
hcp_subdirs[:10]

[S3Path('s3://hcp-openaccess/HCP_1200/100206'),
 S3Path('s3://hcp-openaccess/HCP_1200/100307'),
 S3Path('s3://hcp-openaccess/HCP_1200/100408'),
 S3Path('s3://hcp-openaccess/HCP_1200/100610'),
 S3Path('s3://hcp-openaccess/HCP_1200/101006'),
 S3Path('s3://hcp-openaccess/HCP_1200/101107'),
 S3Path('s3://hcp-openaccess/HCP_1200/101309'),
 S3Path('s3://hcp-openaccess/HCP_1200/101410'),
 S3Path('s3://hcp-openaccess/HCP_1200/101915'),
 S3Path('s3://hcp-openaccess/HCP_1200/102008')]

In [84]:
# Load preprocessed fMRI data
sub115825 = 'shared/HCP/115825/MNINonLinear/Results/rfMRI_REST1_7T_PA/rfMRI_REST1_7T_PA_Atlas_1.6mm_hp2000_clean.dtseries.nii'
subj_bold = ny.load(sub115825)
subj_bold.get_fdata().shape

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


(900, 170494)

In [85]:
# path to MNI space FS atlas
# mni_atlas_fname = 'shared/HCP/115825/MNINonLinear/aparc+aseg.nii.gz'
# mni_atlas = ny.load(mni_atlas_fname)
# np.unique(mni_atlas.get_fdata())

In [86]:
(lh_data_bold, rh_data_bold, subcortex_data_bold) = ny.hcp.cifti_split(subj_bold)

In [87]:
# BOLD signal for every vertex
lh_data_bold.shape

(900, 59292)

In [88]:
rh_data_bold.shape

(900, 59292)

In [89]:
subcortex_data_bold.shape

(900, 62053)

In [90]:
sub115825_labels = 'shared/HCP/115825/MNINonLinear/fsaverage_LR59k/115825.aparc.59k_fs_LR.dlabel.nii'
subj_labels = nib.load(sub115825_labels)
sub_labels = np.unique(subj_labels.get_fdata())
sub_labels

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


array([ 0.,  1.,  2.,  3.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 40.,
       41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53.,
       54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66.,
       67., 68., 69., 70.])

In [91]:
(lh_data_labels, rh_data_labels, subcortex_data_labels) = ny.hcp.cifti_split(subj_labels)
lh_data_labels[0]

array([10., 22., 24., ..., 15., 15., 15.])

In [92]:
subda = ny.hcp_subject(111312)

In [93]:
roi_centers = {}

mesh = subda.hemis['lh_LR59k'].registrations['fs_LR']
for roi in np.unique(lh_data_labels[0]):
    if np.isnan(roi):
        continue
    ii = lh_data_labels[0] == roi 
    roi_coords = mesh.coordinates[:, ii]
    roi_center = np.mean(roi_coords, axis=1)
    closest_vertex = mesh.vertex_hash.query(roi_center)
    roi_centers[roi] = closest_vertex
    

In [95]:
import ipyvolume as ipv
#fig = ny.cortex_plot(subda.hemis['lh_LR59k'], color = lh_data_labels[0])
fig = ny.cortex_plot(subda.hemis['lh_LR59k'], color = 'r', mask = lh_data_labels[0]==33, mesh_alpha=0.5)
#fig = ny.cortex_plot(subda.hemis['rh_LR59k'], color = 'r', mask = lh_data_labels[0]==69, mesh_alpha=0.5)
#fig = ipv.figure()
mesh = subda.hemis['lh_LR59k'].surface('inflated')
(x, y, z) = mesh.coordinates[:, [v for k, v in roi_centers.values()]]
ipv.scatter(x, y, z, marker = 'sphere', size = 5, color = 'blue')
p = ipv.plot([x[0], x[1]], [y[0], y[1]], [z[0], z[1]], color = 'red', size = 5)
#p.size.linewidth = 10
ipv.show()

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.6…

In [67]:
# label ID for every vertex
lh_data_labels.shape
test_idx = np.where(lh_data_labels==5)[0]
np.mean(lh_data_bold[:, test_idx], axis = 1).shape

(900,)

In [78]:
rh_data_labels.shape

(1, 59292)

In [55]:
subcortex_data_labels

In [12]:
# go through every unique label 
# for every label find the vertices with that label
# average BOLD signal for all the vertices
## ==> smaller matrix, 900 x number of labels (about 50 maybe)


In [13]:
# merge BOLD signals and labels together
lh_data = np.vstack((lh_data_bold, lh_data_labels))
lh_data.shape

(901, 59292)

In [107]:
# unique lh label
unique_lh = np.unique(lh_data_labels)
unique_rh = np.unique(rh_data_labels)

In [186]:
### Generate N x T 2D matrix of mean BOLD signal for each subject

avg_lh_bold_mat = []

# loop through unique lh labels
for label in unique_lh[unique_lh != 0]: # for each label that isn't 0
    label_idx = np.where(lh_data_labels == label)[1] # get the index of the vertices that match this label
    label_timeseriesda = np.mean(lh_data_bold[:, label_idx], axis = 1) # take the mean of the vertices within the region
    avg_lh_bold_mat.append(label_timeseriesda) # append the mean BOLD signal of this region at each time point (volume)
avg_lh_bold_mat = np.vstack(avg_lh_bold_mat) # stack all the regions together into a matrix

avg_rh_bold_mat = []

# loop through unique rh labels
for label in unique_rh[unique_rh != 0]: # do the same in the RH
    label_idx = np.where(rh_data_labels == label)[1]
    label_timeseriesda = np.mean(rh_data_bold[:, label_idx], axis = 1)
    avg_rh_bold_mat.append(label_timeseriesda)  
avg_rh_bold_mat = np.vstack(avg_rh_bold_mat) 

avg_bold_mat = np.vstack((avg_lh_bold_mat, avg_rh_bold_mat)) # stack the LH and RH matrices together

(70, 900)

In [187]:
# avg_bold_mat_data = pd.DataFrame(avg_bold_mat)
# avg_bold_mat_data.to_csv('/home/jovyan/test_coneff_input.csv')

In [142]:
### Generate N x N FC matrix for each subject
subj_bold_data = subj_bold.get_fdata()

num_regions = avg_bold_mat.shape[0]
FC_mat = np.zeros((num_regions, num_regions)) # intialize empty matrix

# loop through each region index
for i_region in range(num_regions): # for each region (x-axis)
    for j_region in range(num_regions): # and for each region (y-axis)
        if i_region == j_region: 
            FC_mat == 0 # the correlation between the region and itself is set to 0 by convention
        else: # otherwise compute the pearson correlation between the time series of each region with every other region
            FC_mat[i, j] = pearsonr(subj_bold_data[:, i_region], subj_bold_data[:, j_region])[0]

FC_mat_fisherz = np.arctanh(FC_mat) # to make the distribution of the values more normal, fisher-z transform them

In [142]:
# FC_mat = pd.DataFrame(avg_bold_mat)
# avg_bold_mat_data.to_csv('/home/jovyan/test_coneff_input.csv')